# 01 · Natal Positions (SPEC-B-001)

This notebook boots the ephemeris adapter and computes natal positions for the sample SolarFire exports. The outputs are compared to the golden fixtures that feed later notebooks.

In [1]:
from __future__ import annotations

import json
import math
import os
from pathlib import Path
import pandas as pd
import sys

ASTRO_ROOT = Path(os.environ.get('ASTROENGINE_ROOT', '..')).resolve()
DOCS_ROOT = Path(os.environ.get('DOCS_SITE_ROOT', 'docs-site')).resolve()
FIXTURES = DOCS_ROOT / 'docs' / 'fixtures'
os.environ.setdefault('SE_EPHE_PATH', str(ASTRO_ROOT / 'datasets' / 'swisseph_stub'))
if str(ASTRO_ROOT) not in sys.path:
    sys.path.insert(0, str(ASTRO_ROOT))

In [2]:
events = pd.read_csv(FIXTURES / 'birth_events.csv')
if events.empty:
    raise RuntimeError('Fixture birth_events.csv is empty')

subject = events.iloc[0]
partner = events.iloc[1]
events

,id,name,ts,lat,lon,tz,place
0,subject,Alex,1990-07-11T08:00:00Z,40.7128,-74.0060,America/New_York,"New York, NY"
1,partner,Riley,1992-03-15T20:15:00Z,34.0522,-118.2437,America/Los_Angeles,"Los Angeles, CA"


In [3]:
from astroengine.chart.natal import ChartLocation, compute_natal_chart
from astroengine.ephemeris.swisseph_adapter import SwissEphemerisAdapter

adapter = SwissEphemerisAdapter()

def chart_positions(row):
    moment = pd.to_datetime(row['ts'], utc=True).to_pydatetime()
    loc = ChartLocation(latitude=row['lat'], longitude=row['lon'])
    chart = compute_natal_chart(moment, loc, adapter=adapter)
    data = {name: {'longitude': pos.longitude, 'latitude': pos.latitude, 'distance_au': pos.distance_au} for name, pos in chart.positions.items()}
    return pd.DataFrame(data).T.round(6)

subject_positions = chart_positions(subject)
partner_positions = chart_positions(partner)
subject_positions.head()

,longitude,latitude,distance_au
Sun,108.772824,0.000078,1.016561
Moon,327.028080,1.780025,0.002586
Mercury,118.687552,1.838754,1.300223
Venus,79.421471,-1.136830,1.410495
Mars,29.133036,-2.065950,1.136778


In [4]:
fixture_subject = pd.read_json(FIXTURES / 'positions_subject.json').T
fixture_partner = pd.read_json(FIXTURES / 'positions_partner.json').T

max_delta = (subject_positions['longitude'] - fixture_subject['longitude']).abs().max()
print(f'Max longitude delta subject: {max_delta:.6f}')
max_delta_partner = (partner_positions['longitude'] - fixture_partner['longitude']).abs().max()
print(f'Max longitude delta partner: {max_delta_partner:.6f}')

Max longitude delta subject: 0.000000
Max longitude delta partner: 0.000000


In [5]:
import hashlib
sha = hashlib.sha256((FIXTURES / 'positions_subject.json').read_bytes()).hexdigest()
print('Results checksum:', sha)

Results checksum: 3673f99a953b76ab946d27d2a64d9105d1c3d17c5dff9ef7157a388c781745e7
